In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import torch
from models.layers import TransformerTimeSeriesForecast
import torch.optim as optim 
from torch import nn
from utils.auxiliary_fn import train_one_ep,evaluate_one_ep
from models.LSTM_FCN import LSTM_FCN_2_Atten

# use whole set

In [11]:
# from utils.dataset import Hand_foot_data_with_weath,get_h_f_dataloader,train_val_test_split

# span_len=30
# pre_len=1
# batch_size=16
# dataset=Hand_foot_data_with_weath(
#                                 root_dir_1='./data/case_inf.csv',
#                                 root_dir_2='./data/wea_proed_inf.csv',
#                                 cut_len=span_len,
#                                 slide_win_size=pre_len,
#                                  )
# print(len(dataset))
# train_dataset,val_dataset,test_dataset=train_val_test_split(dataset,train_ratio=0.8,test_ratio=0.1,val_ratio=0.1)

# train_dataloader=get_h_f_dataloader(train_dataset,batch_size,0)
# val_dataloader=get_h_f_dataloader(val_dataset,batch_size,0)
# test_dataloader=get_h_f_dataloader(test_dataset,batch_size,0)

# x,y=next(iter(train_dataloader))
# # print(x)
# print(x.shape)
# print(y.shape)
# # print(y)


# split data

In [12]:
from utils.train_val_test import h_d_train,h_d_val,h_d_test,get_h_f_dataloader

span_len=120
pre_len=1
batch_size=32

train_dataset=h_d_train(cut_len=span_len,slide_win_size=pre_len)
val_dataset=h_d_val(cut_len=span_len,slide_win_size=pre_len)
test_dataset=h_d_test(cut_len=span_len,slide_win_size=pre_len)


train_dataloader=get_h_f_dataloader(train_dataset,batch_size,0)
val_dataloader=get_h_f_dataloader(val_dataset,batch_size,0)
test_dataloader=get_h_f_dataloader(test_dataset,batch_size,0)

print(len(val_dataloader))

x,y=next(iter(train_dataloader))
# print(x)
print(x.shape)
print(y.shape)
print(y)



2
torch.Size([32, 120, 1])
torch.Size([32, 1])
tensor([[ 57.],
        [ 47.],
        [ 85.],
        [ 57.],
        [ 70.],
        [ 56.],
        [ 56.],
        [ 53.],
        [ 64.],
        [ 78.],
        [ 79.],
        [ 84.],
        [ 63.],
        [ 64.],
        [ 49.],
        [ 63.],
        [ 74.],
        [ 82.],
        [ 64.],
        [ 68.],
        [ 59.],
        [ 62.],
        [ 91.],
        [100.],
        [105.],
        [ 91.],
        [ 94.],
        [ 72.],
        [ 89.],
        [ 86.],
        [100.],
        [113.]])


d:\kobe\lib\site-packages\sklearn\preprocessing\_data.py:2583: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
d:\kobe\lib\site-packages\sklearn\preprocessing\_data.py:2583: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(


# try transformer

In [13]:
# model = TransformerTimeSeriesForecast(
#     feature_dim=7,
#     pre_len=pre_len,
#     num_heads=8,
#     hidden_dim=256,
#     num_layers=10,
#     data_len=span_len
#     )

model=LSTM_FCN_2_Atten(
    data_len=span_len,
    pre_len=pre_len,
    input_dim=1,
    hidden_dim=8,
    num_layers=8
)

learning_rate=0.0015
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

# Define the loss function
# loss_fn = nn.SmoothL1Loss()  ## optional_1

from utils.loss_fn.soft_dtw import SoftDTW
# Create the "criterion" object
loss_fn = SoftDTW(use_cuda=False, gamma=0.1)


# Create the data loader
if torch.cuda.is_available():
    device=torch.device('cuda')
else :
    device=torch.device('cpu')
# Train the model

num_epochs = 100


In [14]:
def train_and_evaluate(model, criterion, optimizer, train_loader, evaluate_loader,val_loader,train_fn,test_fn,device,num_epochs):
    """
    Trains and evaluates a PyTorch model for time series forecasting.
    
    Args:
        model: The PyTorch model to train and evaluate.
        criterion: The loss function to use for training and evaluation.
        optimizer: The optimizer to use for training.
        train_loader (torch.utils.data.DataLoader): The data loader to use for training.
        val_loader (torch.utils.data.DataLoader): The data loader to use for evaluation.
        num_epochs (int): The number of epochs to train the model for.
    """
    for epoch in range(num_epochs):
        avg_train_loss=train_fn(optimizer, criterion, train_loader, model,device)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}")
        
        # Evaluation loop
        model.eval()
        val_loss = test_fn(criterion, val_loader, model,device)
        print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}")
    
    # Final evaluation on validation set
    model.eval()
    val_loss = test_fn(criterion, evaluate_loader, model,device)
    print(f"Final Val Loss: {val_loss:.4f}")

In [15]:
train_and_evaluate(
    model=model,
    criterion=loss_fn,
    optimizer=optimizer,
    train_loader=train_dataloader,
    val_loader=val_dataloader,
    evaluate_loader=test_dataloader,
    train_fn=train_one_ep,
    test_fn=evaluate_one_ep,
    device=device,
    num_epochs=num_epochs)